# Import necessary libraries

In [1]:
import numpy as np
import pandas as pd

# Import skeleton from plantscan3d as .txt file (MTG)

In [2]:
# Loading in data
filename = 'two_node_list.txt' #put your filename here

# setting everything up nicely

my_array = pd.read_csv(filename,skiprows=3,sep='\t',usecols=[0,1,3,4,5],names=['Node','Parent','X','Y','Z'])
my_array['Parent'].loc[0] = 0
my_array['Parent'] = my_array['Parent'].astype('int')

# visualize

my_array

E:\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,Node,Parent,X,Y,Z
0,2,0,-0.000024,-0.071371,-0.021333
1,3,2,-0.000070,-0.068443,-0.019051
2,4,3,-0.000042,-0.063910,-0.015813
3,6,4,-0.001965,-0.037631,-0.015186
4,13,6,0.004141,-0.026538,-0.011465
5,22,13,0.009290,-0.017297,-0.004672
6,30,22,0.006994,-0.005217,-0.013308
7,31,22,0.017789,0.001448,-0.013766
8,36,30,0.007520,0.010052,-0.014293
9,41,36,0.013009,0.034582,-0.010445


# Function for calculating branch angles

In [11]:
# function for returning a list of angles

def branch_angles(my_array):
    
    parents = my_array['Parent'] # getting all of the parent nodes
    angles = [] # list of angles for final output
    tested = [] #list of parents already tested

    for parent in parents: # for each parent in the list
        children = my_array[my_array['Parent'] == parent] # retrieve rows of dataframe that list the parent node as their parent
        if len(children) >= 2 and parent not in tested: # if that's 2 or greater, it must be a branching point
            parent_to_child_vectors = [] # list of vectors for branch vectors
            tested.append(parent) # we're testing it now, so add this parent node to the tested list so we don't do it again
            dot_products = [] # making a list of dot products for later
            reference_node_number = my_array[my_array['Node'] == parent]['Parent']
            reference_node_number = reference_node_number.values[0] # we need to first find the node preceding the parent to make a baseline vector
            parent_node_coordinates = np.array([my_array[my_array['Node']==parent]['X'],my_array[my_array['Node']==parent]['Y'],my_array[my_array['Node']==parent]['Z']]) #get the coorddiinates of the parent node
            reference_node_coordinates = np.array([my_array[my_array['Node']==reference_node_number]['X'],my_array[my_array['Node']==reference_node_number]['Y'],my_array[my_array['Node']==reference_node_number]['Z']]) # get the coordinates of the reference node
            reference_to_parent_vector = [parent_node_coordinates - reference_node_coordinates] #subtract to get the vector
            # for each child node, retrieve child node coordinates, make a child vector, calculate intervening branch angle and append to a list to check later
            for i in range(len(children)): 
                child_node_number = children['Node'].iloc[i] 
                child_node_coordinates = np.array([my_array[my_array['Node']==child_node_number]['X'],my_array[my_array['Node']==child_node_number]['Y'],my_array[my_array['Node']==child_node_number]['Z']]) #getting child node coordinates
                parent_to_child_vector = [parent_node_coordinates - child_node_coordinates] # calculating parent to child (or branch) vector
                parent_to_child_vectors.append(parent_to_child_vector) #adding to the list
                
                # this section calculates the dot between each to "branch" vector with the reference vector, with the logic being that the 
                # highest dot product will correspond to the branch vector/reference vector with the highest similarity to one another.
                # We'll treat this as the "axis" we compare against in the proper branch angle calculations
                
                unit_vector_1 = reference_to_parent_vector / np.linalg.norm(reference_to_parent_vector)
                unit_vector_2 = parent_to_child_vector / np.linalg.norm(parent_to_child_vector)
                unit_vector_1 = unit_vector_1.tolist()
                unit_vector_2 = unit_vector_2.tolist()
                unit_vector_1 = [unit_vector_1[0][0][0],unit_vector_1[0][1][0],unit_vector_1[0][2][0]]
                unit_vector_2 = [unit_vector_2[0][0][0],unit_vector_2[0][1][0],unit_vector_2[0][2][0]]
                dot_product = np.dot(unit_vector_1, unit_vector_2)
                dot_products.append(dot_product)
            max_index = np.argmax(dot_products) # finding which dot product is the higest and getting its index
            reference_vector = parent_to_child_vectors[max_index] # retrieving the parent_to_child vector that corresponds to that maximal dot product

            # now that we've established what we're comparing against, we go through each branch vector and compare to this reference (being sure to not report out the 0 degrees in the case where we're comparing the same things)
            for i in range(len(parent_to_child_vectors)): 
                unit_vector_1 = reference_vector / np.linalg.norm(reference_vector)
                unit_vector_2 = parent_to_child_vectors[i] / np.linalg.norm(parent_to_child_vectors[i])
                unit_vector_1 = unit_vector_1.tolist()
                unit_vector_2 = unit_vector_2.tolist()
                unit_vector_1 = [unit_vector_1[0][0][0],unit_vector_1[0][1][0],unit_vector_1[0][2][0]]
                unit_vector_2 = [unit_vector_2[0][0][0],unit_vector_2[0][1][0],unit_vector_2[0][2][0]]
                dot_product = np.dot(unit_vector_1, unit_vector_2)
                
                # getting angle in radians, then converting to degrees
                angle_in_radians = np.arccos(dot_product)
                angle_in_degrees = np.degrees(angle_in_radians)
                if angle_in_degrees != 0: #as long as it's not 0, add and report
                    angles.append(angle_in_degrees)
    return angles

# Retrieve angles from our previously loaded array

In [12]:
angles = branch_angles(my_array)

# Print angles

In [13]:
print(angles)

[32.40380166749817, 66.9839271441569]
